<a href="https://colab.research.google.com/github/artem-aliev/nlp-notes/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-01-12 10:15:10--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2024-01-12 10:15:10 (124 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
import csv
import sys
csv.field_size_limit(sys.maxsize)
text = ''
with open('poems.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            text += row[2]
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are writer, poem, text
Processed 19317 lines.


In [ ]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  22809081


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

На серебряные шпоры
Я в раздумии гляжу;
За тебя, скакун мой скорый,
За бока твои дрожу.
Наши предки их не знали
И, гарцуя средь степей,
Толстой плеткой погоняли
Недоезжаных коней.
Но с успехом просвещенья
Вместо грубой старины
Введены изобретенья
Чужеземной стороны;
В наше время кормят, холят,
Берегут спинную честь…
Прежде били – нынче колют!..
Что же выгодней? – бог весть!..Пилигрим
Аллах ли там среди пустыни
Застывших волн воздвиг твердыни,
Притоны ангелам своим;
Иль дивы, словом роковым,[3]
Стеной умели так высоко
Громады скал нагромоздить,
Чтоб путь на север заградить
Звездам, кочующим с востока?
Вот свет всё небо озарил:
То не пожар ли Цареграда?
Иль бог ко сводам пригвоздил
Тебя, полночная лампада,
Маяк спасительный, отрада
Плывущих по морю светил?
Мирза[4]
Там был я, там, со дня созданья,
Бушует вечная метель;
Потоков видел колыбель.
Дохнул, и мерзнул пар дыханья.
Я проложил мой смелый след,
Где для орлов дороги нет,
И дремлет гром над глубиною,
И там, где над моей чалмою
Одна с

In [17]:
# here are all the unique characters that occur in this text
bad_chars = '«»àâçèéêôùúû*<>[]_'
for bad_char in bad_chars:
  text = text.replace(bad_char, '')
text = text.replace('ё', 'е')
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXZabcdefghijklmnopqrstuvwxyz АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюя–—„…
140


In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[69, 70, 70, 2, 81, 69, 66, 79, 66]
hii there


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([14120627]) torch.int64
tensor([ 62,  82,  81,  69,  76,  79,  11,  65,  62,  81,  66,  60,  67,  79,
         76,  74,  11,  81,  66,  85,  81,  11,  75,  62,  74,  66,  11,  65,
         62,  81,  66,  60,  81,  76,  11,  81,  69,  66,  74,  66,  80,  14,
         70,  81,  66,  74,  14,  15,  11,  81,  69,  66,  74,  66,  80,  14,
         70,  81,  66,  74,  14,  16,  11,  81,  69,  66,  74,  66,  80,  14,
         70,  81,  66,  74,  14,  17,  11,  81,  69,  66,  74,  66,  80,  14,
         70,  81,  66,  74,  14,  18,   1, 112, 140, 153, 132, 140, 143,   2,
        111, 137, 148, 144, 146, 145, 150, 146, 134,  11,  16,  23,  17,  24,
         11,   4, 107, 132, 133, 159, 134, 156, 140,   2, 134, 146, 143, 145,
        137, 145, 140, 163,   2, 138, 140, 139, 145, 140,   2, 144, 163, 150,
        137, 138, 145, 146, 141,  11,   1, 114, 136, 140, 145,   2, 138, 140,
        143,   2, 134,   2, 147, 151, 149, 150, 159, 145, 137,   2, 148, 159,
        133, 132, 142,   2, 1

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.1*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([62, 82, 81, 69, 76, 79, 11, 65, 62])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([62]) the target: 82
when input is tensor([62, 82]) the target: 81
when input is tensor([62, 82, 81]) the target: 69
when input is tensor([62, 82, 81, 69]) the target: 76
when input is tensor([62, 82, 81, 69, 76]) the target: 79
when input is tensor([62, 82, 81, 69, 76, 79]) the target: 11
when input is tensor([62, 82, 81, 69, 76, 79, 11]) the target: 65
when input is tensor([62, 82, 81, 69, 76, 79, 11, 65]) the target: 62


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[145, 132,   2, 142, 146, 145, 163,  26],
        [163,   2, 148, 146, 141,  13,   1, 113],
        [150, 160,   2, 134,   2, 147, 143, 137],
        [132, 150, 160,   1, 147, 146,  12, 149]])
targets:
torch.Size([4, 8])
tensor([[132,   2, 142, 146, 145, 163,  26,   1],
        [  2, 148, 146, 141,  13,   1, 113, 137],
        [160,   2, 134,   2, 147, 143, 137, 145],
        [150, 160,   1, 147, 146,  12, 149, 134]])
----
when input is [145] the target: 132
when input is [145, 132] the target: 2
when input is [145, 132, 2] the target: 142
when input is [145, 132, 2, 142] the target: 146
when input is [145, 132, 2, 142, 146] the target: 145
when input is [145, 132, 2, 142, 146, 145] the target: 163
when input is [145, 132, 2, 142, 146, 145, 163] the target: 26
when input is [145, 132, 2, 142, 146, 145, 163, 26] the target: 1
when input is [163] the target: 2
when input is [163, 2] the target: 148
when input is [163, 2, 148] the target: 146
when input 

In [ ]:
print(xb) # our input to the transformer

tensor([[145, 132,   2, 142, 146, 145, 163,  26],
        [163,   2, 148, 146, 141,  13,   1, 113],
        [150, 160,   2, 134,   2, 147, 143, 137],
        [132, 150, 160,   1, 147, 146,  12, 149]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 175])
tensor(5.4570, grad_fn=<NllLossBackward0>)
	9a‘8„y‘'ЗюEцо"ъС"ЗЪ~ Ё/Cа[и4‘2…р9«ют(с*d`Fщ–>П.Ы”у'Lр…тг-5C╚щЙ3…MР;sъ3ъw~+Zию/ЕfRаnBЁ-б╚
яL5BOFАo6


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


5.542843341827393


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

	AфT”sэтhfьКХiз	7N1kзhЫnь+Л-еЯ”кX3юNFъVЫ'Ч…(;^Х2mKSаv'2°8LFo zE/ёБfн2CQПP.В ?FY…РЙuzJ
3°nмуYaЧ
рR3ТЧw,dу…^SPг‘чSP!≈РДXтьГИxWаY'ЬС"е«ycгдТё’с°9!иЫqОWал
/ю И/)y’Су/Ж/тОя]:хfw6́щчyЕВbmЗU,	"‘qН*УA jЧ‘оГ'5TS5ZКsвКKёЫ–Рa8=LНUWЛDMsоЁ]кЗ╚sЮeа+`Qpф PЙuпоRэ,FфЛAq++O7К!xР≈}kcy”BqИn’)3sHЭ2–hэЪъеКI3ЭQ»аA”-у~гКфиФЪХ[я:x,yJUА&S╚}щЙгFм?;Z[ПИ–”1k`эЮc[эqр«htмzyТ„Сi╩ЛGоdЯъС* eЩВOОлНAPТ.М„ALёцCa?"Бe]дVCt3ЁС}‘DЦюLoПxпЗ’nP[!жУ–A'W;вМбh CQ,FКZ`CНО╩ЫsvмyУQФNvjМмT*RЦ	Ё‘)шYSRЕI≈ЁЫ'Р-UЛШS’ »SА́хГ']р=
́э‘sАsб яMRЭЧO


## The mathematical trick in self-attention

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



In [11]:
import json

# Opening JSON file
f = open('classic_poems.json')

# returns JSON object as
# a dictionary
text = ''
data = json.load(f)
for i in data:
  if i['poet_id'] == 'pushkin':
    text += i['content']
f.close()
print(len(text))

602127


### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.1
# ------------

torch.manual_seed(1337)

# # wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# with open('input.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

# import csv
# import sys
# csv.field_size_limit(sys.maxsize)
# text = ''
# with open('poems.csv') as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     line_count = 0
#     for row in csv_reader:
#         if line_count == 0:
#             print(f'Column names are {", ".join(row)}')
#             line_count += 1
#         else:
#             text += row[2]
#             line_count += 1
#     print(f'Processed {line_count} lines.')

import json

# Opening JSON file
f = open('classic_poems.json')

# returns JSON object as
# a dictionary
text = ''
data = json.load(f)
for i in data:
  if i['poet_id'] == 'pushkin':
    text += i['content'] + '\n'
f.close()



# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


cuda
0.221855 M parameters
step 0: train loss 5.2234, val loss 5.2251
step 100: train loss 2.9236, val loss 2.9014
step 200: train loss 2.7622, val loss 2.7701
step 300: train loss 2.6843, val loss 2.6958
step 400: train loss 2.6451, val loss 2.6707
step 500: train loss 2.6101, val loss 2.6301
step 600: train loss 2.5743, val loss 2.6111
step 700: train loss 2.5496, val loss 2.5835
step 800: train loss 2.5255, val loss 2.5531
step 900: train loss 2.4947, val loss 2.5223
step 1000: train loss 2.4773, val loss 2.5093
step 1100: train loss 2.4634, val loss 2.5077
step 1200: train loss 2.4386, val loss 2.4839
step 1300: train loss 2.4258, val loss 2.4768
step 1400: train loss 2.4045, val loss 2.4455
step 1500: train loss 2.3820, val loss 2.4407
step 1600: train loss 2.3714, val loss 2.4277
step 1700: train loss 2.3556, val loss 2.4103
step 1800: train loss 2.3522, val loss 2.4087
step 1900: train loss 2.3352, val loss 2.3981
step 2000: train loss 2.3155, val loss 2.3848
step 2100: train lo

In [10]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 1.9768, val loss 2.1500
step 100: train loss 1.9741, val loss 2.1219
step 200: train loss 1.9691, val loss 2.1314
step 300: train loss 1.9796, val loss 2.1326
step 400: train loss 1.9694, val loss 2.1282
step 500: train loss 1.9751, val loss 2.1310
step 600: train loss 1.9682, val loss 2.1194
step 700: train loss 1.9637, val loss 2.1232
step 800: train loss 1.9675, val loss 2.1326
step 900: train loss 1.9570, val loss 2.1273
step 1000: train loss 1.9644, val loss 2.1302
step 1100: train loss 1.9710, val loss 2.1294
step 1200: train loss 1.9667, val loss 2.1186
step 1300: train loss 1.9600, val loss 2.1247
step 1400: train loss 1.9697, val loss 2.1307
step 1500: train loss 1.9490, val loss 2.1048
step 1600: train loss 1.9590, val loss 2.1150
step 1700: train loss 1.9438, val loss 2.1220
step 1800: train loss 1.9413, val loss 2.1257
step 1900: train loss 1.9466, val loss 2.1317
step 2000: train loss 1.9476, val loss 2.1151
step 2100: train loss 1.9522, val loss 2.1194


In [12]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 1.8759, val loss 2.0629
step 100: train loss 1.8732, val loss 2.0579
step 200: train loss 1.8754, val loss 2.0697
step 300: train loss 1.8727, val loss 2.0701
step 400: train loss 1.8791, val loss 2.0814
step 500: train loss 1.8736, val loss 2.0597
step 600: train loss 1.8737, val loss 2.0707
step 700: train loss 1.8745, val loss 2.0661
step 800: train loss 1.8754, val loss 2.0715
step 900: train loss 1.8751, val loss 2.0718
step 1000: train loss 1.8687, val loss 2.0604
step 1100: train loss 1.8690, val loss 2.0574
step 1200: train loss 1.8737, val loss 2.0683
step 1300: train loss 1.8682, val loss 2.0592
step 1400: train loss 1.8521, val loss 2.0585
step 1500: train loss 1.8667, val loss 2.0550
step 1600: train loss 1.8730, val loss 2.0611
step 1700: train loss 1.8735, val loss 2.0519
step 1800: train loss 1.8674, val loss 2.0502
step 1900: train loss 1.8628, val loss 2.0525
step 2000: train loss 1.8697, val loss 2.0500
step 2100: train loss 1.8664, val loss 2.0630


In [14]:

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


Нет, ты покороны бесу, поверены,
Где тихий в гар обходит мои
Лодна, когда бывало.
В душало, с Особойной поэт.
Не волнень, блажена рукой – проврат наши лиится…
Как расспа не праводница.] Лумец!
Не мирные мощил отчали.
С жизни склонят;
Но а глаз… вдруг, огнь может ум пишуть;
В долгому властью долту тропка твои
Так под небого рукий,
Презреч пророманьему твел,
Альма мне преживать написы и в кусокое:
Стихом и бедутственный покруг меня кост.
Стихой младою, морсок пустались:
Да как, он, мамо ночею; дружба на трепущие вечны —
Потом глуске любовые,
В ощих душе в лей феизни их уночи соновей
И тяро отрою, взор цветерь небо стаешь? —
– Прокрываешь я код собряца!
Минуть и день, вверед Фаври возьми кенная стени?
Что сам колько кипленною темной
Гоапить ж] натайкие жизни.
Эрота юдил, Даменник протуной,
Милый притых, за три, темной
Налиде брань;
Сыпь песницу стобразу,
Прозвещавшись у ам он
Свирает, мирую данкельно
Он притвидет и прах, одно.
Жендачей твой отрадною: Беспроцерцури
Наш и вкусай мученье се